In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import scipy.stats as ss, scipy.integrate as integrate

In [2]:
df = pd.read_csv('data/policecalls2021.csv')
df

,CDTS,EID,START_DATE,CALL_NUMBER,PRIORITY,REPORT_DATE,OFFENSE_DATE,OFFENSE_TIME,CALLTYPE_CODE,CALL_TYPE,FINAL_DISPO_CODE,FINAL_DISPO,COMMON_PLACE_NAME,ADDRESS,CITY,STATE
0,20210101000447PS,8444832,5/15/2021 12:00:00 AM,P210010001,4,1/1/2021 12:00:00 AM,1/1/2021 12:00:00 AM,00:00:08,415FC,"DISTURBANCE, FIRECRACKERS",CAN,Canceled,NaN,[0]-[100] ARCHER ST,San Jose,CA
1,20210101000117PS,8444833,5/15/2021 12:00:00 AM,P210010002,4,1/1/2021 12:00:00 AM,1/1/2021 12:00:00 AM,00:01:05,415FC,"DISTURBANCE, FIRECRACKERS",CAN,Canceled,NaN,[6200]-[6300] MOUNTFORD DR,San Jose,CA
2,20210101000340PS,8444839,5/15/2021 12:00:00 AM,P210010006,4,1/1/2021 12:00:00 AM,1/1/2021 12:00:00 AM,00:03:12,415FC,"DISTURBANCE, FIRECRACKERS",CAN,Canceled,87 AT BRANHAM,NaN,San Jose,CA
3,20210102001021PS,8444842,5/15/2021 12:00:00 AM,P210010008,3,1/1/2021 12:00:00 AM,1/1/2021 12:00:00 AM,00:03:47,602PC,TRESPASSING,CAN,Canceled,NaN,Not a valid geographical locat,San Jose,CA
4,20210101000547PS,8444844,5/15/2021 12:00:00 AM,P210010010,4,1/1/2021 12:00:00 AM,1/1/2021 12:00:00 AM,00:05:02,415FC,"DISTURBANCE, FIRECRACKERS",CAN,Canceled,NaN,BLOSSOM HILL RD & MONTEREY FRO,San Jose,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375892,20211227132835PS,8907430,1/1/2022 12:00:00 AM,P213610308,2,12/27/2021 12:00:00 AM,12/27/2021 12:00:00 AM,11:37:54,WELCK,WELFARE CHECK,N,No report required; dispatch record only,NaN,[1900]-[2000] ROSENELFE CL,San Jose,CA
1375893,20211227132931PS,8907572,1/1/2022 12:00:00 AM,P213610428,6,12/27/2021 12:00:00 AM,12/27/2021 12:00:00 AM,13:23:09,1195,VEHICLE STOP,D,"Traffic Citation Issued, Hazardous Violation",NaN,CAMDEN AV & MT FOREST DR,San Jose,CA
1375894,20211227133017PS,8907496,1/1/2022 12:00:00 AM,P213610363,5,12/27/2021 12:00:00 AM,12/27/2021 12:00:00 AM,12:28:47,10851R,RECOVERED STOLEN VEHICLE,CAN,Canceled,NaN,BERNAL RD & SAN IGNACIO AV,San Jose,CA
1375895,20211227133033PS,8907578,1/1/2022 12:00:00 AM,P213610433,2,12/27/2021 12:00:00 AM,12/27/2021 12:00:00 AM,13:29:27,415,DISTURBANCE,CAN,Canceled,NaN,[300]-[400] S 1ST ST,San Jose,CA


In [3]:
calltype_codes_with_text = df[['CALLTYPE_CODE', 'CALL_TYPE']].drop_duplicates()
calltype_codes_with_text.to_csv('calltype_codes.csv')

In [4]:
calltype_codes_with_text

,CALLTYPE_CODE,CALL_TYPE
0,415FC,"DISTURBANCE, FIRECRACKERS"
3,602PC,TRESPASSING
11,1033A,"ALARM, AUDIBLE"
14,1057,FIREARMS DISCHARGED
17,1033S,"ALARM, SILENT"
...,...,...
826877,211AG,"ARMED ROBBERY, GANG RELATED"
1078671,300,W&I-UNDER JURIS OF JUV COURT
1105956,1095C,PEDESTRIAN STOP-CURFEW VIOL
1270839,601T,TRUANT (TABS)


In [5]:
dispo_codes_with_text = df[['FINAL_DISPO_CODE', 'FINAL_DISPO']].drop_duplicates()
dispo_codes_with_text = dispo_codes_with_text.sort_values(by=['FINAL_DISPO_CODE'], axis=0)
dispo_codes_with_text.to_csv('dispo_codes.csv')

In [6]:
dispo_codes_with_text

,FINAL_DISPO_CODE,FINAL_DISPO
109,A,Arrest Made
1353,ADV,No Disposition
666,B,Arrest by Warrant
304,C,Criminal Citation
0,CAN,Canceled
109328,D,"Traffic Citation Issued, Hazardous Violation"
64,D,"Traffic Citation Issued, Hazar"
41,DUPNCAN,No Disposition
109408,E,"Traffic Citation Issued, Non-Hazardous Violation"
120,E,"Traffic Citation Issued, Non-H"
